In [ ]:
## set covid19_datasciente as path for python find bulletin package
import sys
from pathlib import Path
from os import getcwd, remove, walk
from os.path import join, basename, isdir
sys.path.append(str(Path(getcwd()).parent.parent))

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)

import codecs
import numpy as np
from glob import glob
from epiweeks import Week

In [ ]:
from bulletin import default_input, default_output, tables_path, root
default_output = join(default_output,'vacinacao')

from bulletin.systems import Notifica, CasosConfirmados, eSUS, Sivep
from bulletin.utils.static import Municipios
from bulletin.utils.xls_ import fit_cols
from bulletin.utils.normalize import normalize_text, normalize_hash, date_hash

from datetime import datetime, date, timedelta

hoje = pd.to_datetime(date.today())
ontem = hoje - timedelta(1)
anteontem = ontem - timedelta(1)
data_retroativos = ontem - timedelta(31)

In [ ]:
situacao_labels = { 0:'Não Vacinado', 1:'Esquema Primário Incompleto', 2:'Esquema Primário Completo ', 3: 'Esquema Primário Completo + Reforço'}
mes_labels = {key+1:value for key,value in enumerate(['Janeiro','Fevereiro','Março','Abril','Maio','Junho','Julho','Agosto','Setembro','Outubro','Novembro','Dezembro'])}

In [ ]:
## CARREGAMENTO DOS DADOS

In [ ]:
vac = eSUS('vacinados')
vac.load()
vacinados = vac.df

In [ ]:
cc_vacinados = CasosConfirmados('cc_vacinados')
cc_vacinados.load()
casos = cc_vacinados.df

In [ ]:
casos.columns

In [ ]:
municipios = Municipios()
municipios['mun_resid'] = municipios['municipio']
municipios.loc[municipios['uf']!='PR','mun_resid'] = municipios.loc[municipios['uf']!='PR','municipio'] + '/' + municipios['uf']

In [ ]:
## TIRAR DO BANCO AQUELES QUE MORRERAM VACINADOS

In [ ]:
#extração dos óbitos que tiveram pelo mennos uma dose de vacina aplicada
obitos = casos.loc[(casos['evolucao']==2)]
obitos_retirar = obitos.loc[obitos['cns'].notna()]

In [ ]:
vacinados.set_index('cns', inplace=True)
obitos_retirar.set_index('cns', inplace=True)

In [ ]:
vacinados = vacinados.drop(index=obitos_retirar.loc[obitos_retirar.index.isin(vacinados.index)].index)

In [ ]:
vacinados.reset_index(inplace=True)
obitos_retirar.reset_index(inplace=True)

In [ ]:
## FAZER A EXTRAÇÃO DA RELAÇÃO DE PESSOAS

In [ ]:
vacinados.columns

In [ ]:
vacinados['dias_apos_ultima_dose'] = (vacinados['data_ultima_dose'] - hoje).dt.days * -1

In [ ]:
pfizer_days = 120
corona_days = 120
astra_days = 120
janssen_days = 120
# df_final = vacinados.loc[vacinados['situacao'] != 'inconsistencia'].copy()
df_final = vacinados.copy()

In [ ]:
pfizer = df_final.loc[(df_final['vacina_nome_2a_dose'] == 'PFIZER') & (df_final['situacao_atual'] == 2)]
pfizer = pfizer.loc[pfizer['dias_apos_ultima_dose'] > pfizer_days]

In [ ]:
corona = df_final.loc[(df_final['vacina_nome_2a_dose'] == 'CORONAVAC') & (df_final['situacao_atual'] == 2)]
corona = corona.loc[corona['dias_apos_ultima_dose'] > corona_days]

In [ ]:
astra = df_final.loc[(df_final['vacina_nome_2a_dose'] == 'ASTRAZENECA') & (df_final['situacao_atual'] == 2)]
astra = astra.loc[astra['dias_apos_ultima_dose'] > astra_days]

In [ ]:
janssen = df_final.loc[(df_final['vacina_nome_dose_unica'] == 'JANSSEN') & (df_final['situacao_atual'] == 2)]
janssen = janssen.loc[janssen['dias_apos_ultima_dose'] > janssen_days]

In [ ]:
df_faltosos = pd.concat([pfizer, corona, astra,janssen])

In [ ]:
df_faltosos['vacina_nome'] = df_faltosos['vacina_nome_2a_dose'] 
df_faltosos.loc[df_faltosos['vacina_nome_dose_unica'].notna(),'vacina_nome'] = df_faltosos.loc[df_faltosos['vacina_nome_dose_unica'].notna(),'vacina_nome_dose_unica'] 

In [ ]:
df_faltosos = pd.merge(df_faltosos.rename(columns={'ibge_atendimento':'ibge'}),municipios[['ibge','macro','rs','uf','municipio','regional']],on='ibge',how='left').rename(columns={'ibge':'ibge_residencia'})

In [ ]:
# POR FAIXA ETÁRIA
df_faltosos.loc[(df_faltosos['idade'] >= 5) & (df_faltosos['idade'] < 12), 'faixa_etaria'] = '5-11 anos'
df_faltosos.loc[(df_faltosos['idade'] >= 12) & (df_faltosos['idade'] < 18), 'faixa_etaria'] = '12-17 anos'
df_faltosos.loc[(df_faltosos['idade'] >= 18) & (df_faltosos['idade'] <= 19), 'faixa_etaria'] = '18-19 anos'
df_faltosos.loc[(df_faltosos['idade'] >= 20) & (df_faltosos['idade'] <= 24), 'faixa_etaria'] = '20-24 anos'
df_faltosos.loc[(df_faltosos['idade'] >= 25) & (df_faltosos['idade'] <= 29), 'faixa_etaria'] = '25-29 anos'
df_faltosos.loc[(df_faltosos['idade'] >= 30) & (df_faltosos['idade'] <= 34), 'faixa_etaria'] = '30-34 anos'
df_faltosos.loc[(df_faltosos['idade'] >= 35) & (df_faltosos['idade'] <= 39), 'faixa_etaria'] = '35-39 anos'
df_faltosos.loc[(df_faltosos['idade'] >= 40) & (df_faltosos['idade'] <= 44), 'faixa_etaria'] = '40-44 anos'
df_faltosos.loc[(df_faltosos['idade'] >= 45) & (df_faltosos['idade'] <= 49), 'faixa_etaria'] = '45-49 anos'
df_faltosos.loc[(df_faltosos['idade'] >= 50) & (df_faltosos['idade'] <= 54), 'faixa_etaria'] = '50-54anos'
df_faltosos.loc[(df_faltosos['idade'] >= 55) & (df_faltosos['idade'] <= 59), 'faixa_etaria'] = '55-59 anos'
df_faltosos.loc[(df_faltosos['idade'] >= 60) & (df_faltosos['idade'] <= 64), 'faixa_etaria'] = '60-64 anos'
df_faltosos.loc[(df_faltosos['idade'] >= 65) & (df_faltosos['idade'] <= 69), 'faixa_etaria'] = '65-69 anos'
df_faltosos.loc[(df_faltosos['idade'] >= 70) & (df_faltosos['idade'] <= 74), 'faixa_etaria'] = '70-74 anos'
df_faltosos.loc[(df_faltosos['idade'] >= 75) & (df_faltosos['idade'] <= 79), 'faixa_etaria'] = '75-79 anos'
df_faltosos.loc[(df_faltosos['idade'] >= 80), 'faixa_etaria'] = '>80 anos'

In [ ]:
df_faltosos = df_faltosos.loc[df_faltosos['rs'] != 99]
df_faltosos = df_faltosos.loc[df_faltosos['idade'] >= 18]

In [ ]:
archive = join(default_output, 'demanda_FALTOSOS(reforco).xlsx')


writer = pd.ExcelWriter(archive,
                    engine='xlsxwriter',
                    datetime_format='dd/mm/yyyy',
                    date_format='dd/mm/yyyy')

In [ ]:
tabela = df_faltosos.groupby(['vacina_nome'])[['cns']].count().reset_index()
tabela = tabela.rename(columns={'vacina_nome':'VACINA NOME', 'cns':'CONTAGEM'})
tabela.loc['TOTAL', ['VACINA NOME','CONTAGEM']] = ('TOTAL PR', tabela['CONTAGEM'].sum())
tabela['CONTAGEM'] = tabela['CONTAGEM'].astype(int)
tabela

In [ ]:
tabela.to_excel(writer, 'totais', index=False)

In [ ]:
tabela = df_faltosos.groupby(['rs'])[['cns']].count().reset_index()
tabela = tabela.rename(columns={'rs':'RS', 'cns':'CONTAGEM'})
tabela.loc['TOTAL', ['RS','CONTAGEM']] = ('TOTAL PR', tabela['CONTAGEM'].sum())
tabela['CONTAGEM'] = tabela['CONTAGEM'].astype(int)
tabela

In [ ]:
tabela.to_excel(writer, 'por_rs', index=False)

In [ ]:
tabela = df_faltosos.groupby(['vacina_nome', 'rs'])[['cns']].count()
tabela = tabela.rename(columns={'cns':'CONTAGEM'})
tabela.index.names = ['VACINA NOME', 'RS']
tabela.loc['TOTAL', ['CONTAGEM']] = (tabela['CONTAGEM'].sum())
tabela['CONTAGEM'] = tabela['CONTAGEM'].astype(int)
tabela

In [ ]:
tabela.to_excel(writer, 'por_rs_vacina', index=True)

In [ ]:
tabela = df_faltosos.groupby(['rs', 'municipio'])[['cns']].count()
tabela = tabela.rename(columns={'cns':'CONTAGEM'})
tabela.index.names = ['RS', 'MUNICÍPIO']
tabela.loc['TOTAL', ['CONTAGEM']] = (tabela['CONTAGEM'].sum())
tabela['CONTAGEM'] = tabela['CONTAGEM'].astype(int)
tabela

In [ ]:
tabela.to_excel(writer, 'por_municipio', index=True)

In [ ]:
tabela = df_faltosos.groupby(['vacina_nome', 'rs', 'municipio'])[['cns']].count()
tabela = tabela.rename(columns={'cns':'CONTAGEM'})
tabela.index.names = ['VACINA NOME', 'RS', 'MUNICÍPIO']
tabela.loc['TOTAL', ['CONTAGEM']] = (tabela['CONTAGEM'].sum())
tabela['CONTAGEM'] = tabela['CONTAGEM'].astype(int)
tabela

In [ ]:
tabela.to_excel(writer, 'por_municipio_vacina', index=True)

In [ ]:
tabela = df_faltosos.groupby(['vacina_nome', 'faixa_etaria'])[['cns']].count()
tabela = tabela.rename(columns={'cns':'ÚLTIMA DOSE APLICADA'})
tabela.index.names = ['VACINA NOME', 'FAIXA ETÁRIA']
tabela = tabela.unstack('VACINA NOME')
tabela['TOTAL'] = tabela.sum(axis=1)

In [ ]:
tabela.to_excel(writer, 'faixa-etaria', index=True)

In [ ]:
writer.save()
writer.close()

In [ ]:
# df_faltosos.to_excel(join(default_output, 'faltosos_nominais.xlsx'))